In [3]:
import pdb

In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torchtext.vocab import GloVe
from torchtext.data import Field, BucketIterator
from torchtext.data.metrics import bleu_score
import spacy

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
#Reading files

with open('trainen.txt', encoding='utf8') as f:
    eng_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('trainta.txt', encoding='utf8') as f:
    tamil_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('deven.txt', encoding='utf8') as f:
    eng_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('devta.txt', encoding='utf8') as f:
    tamil_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
embedding_glove = GloVe(name='6B', dim=100)

spacy_en = spacy.load('en_core_web_sm')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

ENG = Field(tokenize = tokenize_en, init_token='sos', eos_token = 'eos', lower=True)
processed_eng_train = list(map(lambda x: ENG.preprocess(x), eng_train))
processed_eng_test = list(map(lambda x: ENG.preprocess(x), eng_test))

ENG.build_vocab(processed_eng_train, vectors=embedding_glove)

In [8]:
def preprocess(processed_eng):
    
    #function to return the numericalized version of the tokenized sentences
    X = []
    for tokenized_sentence in processed_eng:
        int_sequence = [2]  #first element is the SOS token 
        for token in tokenized_sentence:
            int_sequence.append(ENG.vocab.stoi[token])
        int_sequence.append(3) #last element is the EOS token
        X.append(int_sequence)
    
    return X

# X_train and X_test are lists of lists with the integer sequences for a given sentence
X_train = preprocess(processed_eng_train)
X_test = preprocess(processed_eng_test)

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Same thing for Tamil sentences
TAM = Tokenizer()
TAM.fit_on_texts(tamil_train)
Y_train = TAM.texts_to_sequences(tamil_train)
Y_test = TAM.texts_to_sequences(tamil_test)

#adding EOS token
_ = [y.append(0) for y in Y_train]
_ = [y.append(0) for y in Y_test]

Using TensorFlow backend.


In [10]:
source_vocab_size = len(ENG.vocab)
target_vocab_size = len(TAM.word_index)+2
print(source_vocab_size)
print(target_vocab_size)

9736
18670


In [17]:
class Encoder(nn.Module):
    
    def __init__(self, embed_size, hidden_size, ENG):
        
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding.from_pretrained(ENG.vocab.vectors)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        
    def forward(self, x, enc_hidden):
        
        enc_output, enc_hidden = self.lstm(self.embed(x).view(1,1,-1), enc_hidden)
        return enc_output, enc_hidden

In [18]:
class Decoder(nn.Module):
    
    def __init__(self, embed_size, hidden_size, n_classes):
        
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(n_classes, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        self.fc = nn.Linear(hidden_size, n_classes)
        self.logsoftmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x, dec_hidden):
        
        dec_output, dec_hidden = self.lstm(self.embed(x).view(1,1,-1), dec_hidden)
        dec_output = self.logsoftmax(self.fc(dec_output[0]))
        return dec_output, dec_hidden

In [19]:
def loss_per_pair(source_sentence, target_sentence, enc_obj, dec_obj, enc_optimiser, dec_optimiser, loss_fn):
    
    Ts = source_sentence.size(-1)
    Tt = target_sentence.size(-1)
    enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))
    
    enc_optimiser.zero_grad()
    dec_optimiser.zero_grad()
    
    loss_val = 0
    
    for i in range(Ts):
        enc_output, enc_hidden = enc_obj(source_sentence[i], enc_hidden)
    
    dec_input = torch.tensor([[target_vocab_size-1]], device=device)  #SOS token
    
    # first hidden state of decoder is made the final hidden state of the encoder and cell state is initialised with zeros
    dec_hidden = (enc_hidden[0], torch.zeros(1, 1, dec_obj.hidden_size, device=device))
    
    for i in range(Tt):
        
        dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
        _ , index = dec_output.topk(1)
        dec_input = index.squeeze().detach()
            
        target_word = torch.cuda.LongTensor([target_sentence[i].item()])
        
        loss_val += loss_fn(dec_output, target_word)
        
        if dec_input.item() == 0:
            break

    loss_val.backward()
    
    nn.utils.clip_grad_norm_(enc_obj.parameters(), 0.5, 1)
    nn.utils.clip_grad_norm_(dec_obj.parameters(), 0.5, 1)

    enc_optimiser.step()
    dec_optimiser.step()

    return loss_val.item()/Tt

In [20]:
def train_model(sources, targets, enc_obj, dec_obj, lr=0.001, mom=0.9):
    
    loss_fn = nn.NLLLoss()
    enc_optimiser = optim.SGD(enc_obj.parameters(), lr=lr, momentum=mom)
    dec_optimiser = optim.SGD(dec_obj.parameters(), lr=lr, momentum=mom)
#     enc_optimiser = optim.Adagrad(enc_obj.parameters())
#     dec_optimiser = optim.Adagrad(dec_obj.parameters())
    
    max_epochs = 100
    old_loss = np.inf
    indices = [i for i in range(len(sources))]
    
    for epoch in range(max_epochs):
        
        # shuffling the input data manually at the start of every new epoch
        np.random.shuffle(indices)
        sources = list(np.array(sources)[indices])
        targets = list(np.array(targets)[indices])
        
        running_loss = 0.0
        num_sentences = len(sources)
        
        for i in range(num_sentences):
            source_sentence = torch.cuda.LongTensor(sources[i])
            target_sentence = torch.cuda.LongTensor(targets[i])

            loss = loss_per_pair(source_sentence, target_sentence, enc_obj, dec_obj, enc_optimiser, dec_optimiser, loss_fn)
            running_loss += loss

            if i%int(num_sentences*0.1)==0:
                print("Epoch", epoch+1, ":", (i/int(num_sentences*0.1))*10,'% done')
                print("Current loss:", running_loss)
                
        print('\nEpoch', epoch+1,"\n")
        print("Encoder obj lstm wts sum=", torch.sum(enc_obj.lstm.weight_ih_l0))
        print("Encoder obj lstm bias sum=", torch.sum(enc_obj.lstm.bias_ih_l0))
        print('Decoder obj emb wts sum=', torch.sum(dec_obj.embed.weight.data))
        print('Decoder obj lstm wts sum=', torch.sum(dec_obj.lstm.weight_ih_l0.data))
        print('Decoder obj lstm bias sum=', torch.sum(dec_obj.lstm.bias_ih_l0.data))
        print('Decoder obj linear wts sum=', torch.sum(dec_obj.fc.weight.data))
        print('Decoder obj linear bias sum=', torch.sum(dec_obj.fc.bias.data))
        
        
        if abs(running_loss-old_loss)/running_loss < 1e-3:
            print('Converged')
            break
    
        old_loss = running_loss

    print("Finished Training")

In [26]:
embedding_size = 100
hidden_size = 150
encoder = Encoder(embedding_size, hidden_size, ENG).to(device)
decoder = Decoder(embedding_size, hidden_size, target_vocab_size).to(device)
train_model(X_train, Y_train, encoder, decoder)

Epoch 1 : 0.0 % done
Current loss: 9.899867375691732
Epoch 1 : 10.0 % done
Current loss: 25720.718851363305
Epoch 1 : 20.0 % done
Current loss: 51394.15464118719
Epoch 1 : 30.0 % done
Current loss: 69146.16873875647
Epoch 1 : 40.0 % done
Current loss: 85745.7463004592
Epoch 1 : 50.0 % done
Current loss: 98106.70397996604
Epoch 1 : 60.0 % done
Current loss: 110597.1368698363
Epoch 1 : 70.0 % done
Current loss: 123137.70376641808
Epoch 1 : 80.0 % done
Current loss: 135578.30092511413
Epoch 1 : 90.0 % done
Current loss: 148111.68201611942
Epoch 1 : 100.0 % done
Current loss: 160678.32964719375

Epoch 1 

Encoder obj lstm wts sum= tensor(-12.5194, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(-2.0034, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(-938.9765, device='cuda:0')
Decoder obj lstm wts sum= tensor(27.7148, device='cuda:0')
Decoder obj lstm bias sum= tensor(2.8341, device='cuda:0')
Decoder obj linear wts sum= tensor(1.4557, de

Epoch 9 : 0.0 % done
Current loss: 8.895622253417969
Epoch 9 : 10.0 % done
Current loss: 12013.574779326655
Epoch 9 : 20.0 % done
Current loss: 23861.65538287842
Epoch 9 : 30.0 % done
Current loss: 35805.297238324434
Epoch 9 : 40.0 % done
Current loss: 47716.26466008697
Epoch 9 : 50.0 % done
Current loss: 59543.998930705966
Epoch 9 : 60.0 % done
Current loss: 71428.71631530761
Epoch 9 : 70.0 % done
Current loss: 83105.21379483763
Epoch 9 : 80.0 % done
Current loss: 95044.0367712256
Epoch 9 : 90.0 % done
Current loss: 106869.53338089182
Epoch 9 : 100.0 % done
Current loss: 118725.74222133866

Epoch 9 

Encoder obj lstm wts sum= tensor(-11.0364, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(-1.0150, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(-938.9003, device='cuda:0')
Decoder obj lstm wts sum= tensor(34.0505, device='cuda:0')
Decoder obj lstm bias sum= tensor(3.6049, device='cuda:0')
Decoder obj linear wts sum= tensor(9.3065, de

Epoch 17 : 0.0 % done
Current loss: 4.86545991897583
Epoch 17 : 10.0 % done
Current loss: 10856.255534485854
Epoch 17 : 20.0 % done
Current loss: 21713.314872554834
Epoch 17 : 30.0 % done
Current loss: 32466.788709255597
Epoch 17 : 40.0 % done
Current loss: 43293.644785765544
Epoch 17 : 50.0 % done
Current loss: 54046.71659392064
Epoch 17 : 60.0 % done
Current loss: 64858.79185280345
Epoch 17 : 70.0 % done
Current loss: 75738.95720282312
Epoch 17 : 80.0 % done
Current loss: 86578.87020805229
Epoch 17 : 90.0 % done
Current loss: 97232.24801376095
Epoch 17 : 100.0 % done
Current loss: 108014.23123478233

Epoch 17 

Encoder obj lstm wts sum= tensor(-8.4984, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(1.5584, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(-938.7200, device='cuda:0')
Decoder obj lstm wts sum= tensor(44.7524, device='cuda:0')
Decoder obj lstm bias sum= tensor(4.9392, device='cuda:0')
Decoder obj linear wts sum= tensor(

Encoder obj lstm bias sum= tensor(2.7995, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(-938.6371, device='cuda:0')
Decoder obj lstm wts sum= tensor(50.2873, device='cuda:0')
Decoder obj lstm bias sum= tensor(5.5861, device='cuda:0')
Decoder obj linear wts sum= tensor(22.9325, device='cuda:0')
Decoder obj linear bias sum= tensor(7.0477, device='cuda:0')
Epoch 25 : 0.0 % done
Current loss: 3.817983627319336
Epoch 25 : 10.0 % done
Current loss: 10530.584657241781
Epoch 25 : 20.0 % done
Current loss: 20741.245131078995
Epoch 25 : 30.0 % done
Current loss: 31054.96082677888
Epoch 25 : 40.0 % done
Current loss: 41250.223298694036
Epoch 25 : 50.0 % done
Current loss: 51610.050029725826
Epoch 25 : 60.0 % done
Current loss: 61887.78359559428
Epoch 25 : 70.0 % done
Current loss: 72260.94040141019
Epoch 25 : 80.0 % done
Current loss: 82484.05960980369
Epoch 25 : 90.0 % done
Current loss: 92648.89387256415
Epoch 25 : 100.0 % done
Current loss: 102975.68703746048

Epoch 

Epoch 32 : 40.0 % done
Current loss: 40100.70013252021
Epoch 32 : 50.0 % done
Current loss: 50152.183334438545
Epoch 32 : 60.0 % done
Current loss: 60222.146152396235
Epoch 32 : 70.0 % done
Current loss: 70328.29315219985
Epoch 32 : 80.0 % done
Current loss: 80467.36920685862
Epoch 32 : 90.0 % done
Current loss: 90605.3699381148
Epoch 32 : 100.0 % done
Current loss: 100665.6635818304

Epoch 32 

Encoder obj lstm wts sum= tensor(-7.6895, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(3.7705, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(-938.5756, device='cuda:0')
Decoder obj lstm wts sum= tensor(55.1264, device='cuda:0')
Decoder obj lstm bias sum= tensor(6.1002, device='cuda:0')
Decoder obj linear wts sum= tensor(29.1930, device='cuda:0')
Decoder obj linear bias sum= tensor(8.9293, device='cuda:0')
Epoch 33 : 0.0 % done
Current loss: 6.485856374104817
Epoch 33 : 10.0 % done
Current loss: 9936.873499194588
Epoch 33 : 20.0 % done
Cur

Epoch 40 : 10.0 % done
Current loss: 10054.633222509397
Epoch 40 : 20.0 % done
Current loss: 19880.4832888145
Epoch 40 : 30.0 % done
Current loss: 29692.73399864459
Epoch 40 : 40.0 % done
Current loss: 39511.10680620507
Epoch 40 : 50.0 % done
Current loss: 49300.622504963656
Epoch 40 : 60.0 % done
Current loss: 59039.84047621319
Epoch 40 : 70.0 % done
Current loss: 68932.32235499058
Epoch 40 : 80.0 % done
Current loss: 78836.36584571861
Epoch 40 : 90.0 % done
Current loss: 88645.13603494983
Epoch 40 : 100.0 % done
Current loss: 98557.09116401123

Epoch 40 

Encoder obj lstm wts sum= tensor(-7.7206, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(4.6937, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(-938.5217, device='cuda:0')
Decoder obj lstm wts sum= tensor(59.6093, device='cuda:0')
Decoder obj lstm bias sum= tensor(6.5780, device='cuda:0')
Decoder obj linear wts sum= tensor(34.6256, device='cuda:0')
Decoder obj linear bias sum= te

Encoder obj lstm bias sum= tensor(5.1827, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(-938.4931, device='cuda:0')
Decoder obj lstm wts sum= tensor(62.5875, device='cuda:0')
Decoder obj lstm bias sum= tensor(6.8866, device='cuda:0')
Decoder obj linear wts sum= tensor(38.7637, device='cuda:0')
Decoder obj linear bias sum= tensor(12.1423, device='cuda:0')
Epoch 48 : 0.0 % done
Current loss: 6.743631998697917
Epoch 48 : 10.0 % done
Current loss: 9719.567335931199
Epoch 48 : 20.0 % done
Current loss: 19509.815793000645
Epoch 48 : 30.0 % done
Current loss: 29160.707861669132
Epoch 48 : 40.0 % done
Current loss: 38862.503019794465
Epoch 48 : 50.0 % done
Current loss: 48634.08500087948
Epoch 48 : 60.0 % done
Current loss: 58453.80622109232
Epoch 48 : 70.0 % done
Current loss: 68051.18531782312
Epoch 48 : 80.0 % done
Current loss: 77835.67995203029
Epoch 48 : 90.0 % done
Current loss: 87447.42581337591
Epoch 48 : 100.0 % done
Current loss: 97252.701854923

Epoch 48 

Epoch 55 : 40.0 % done
Current loss: 38484.35871985605
Epoch 55 : 50.0 % done
Current loss: 48132.07652584304
Epoch 55 : 60.0 % done
Current loss: 57938.959101279106
Epoch 55 : 70.0 % done
Current loss: 67511.92819381118
Epoch 55 : 80.0 % done
Current loss: 77091.20640990863
Epoch 55 : 90.0 % done
Current loss: 86749.04883800462
Epoch 55 : 100.0 % done
Current loss: 96492.81923931219

Epoch 55 

Encoder obj lstm wts sum= tensor(-8.6350, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(5.4457, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(-938.4770, device='cuda:0')
Decoder obj lstm wts sum= tensor(65.0369, device='cuda:0')
Decoder obj lstm bias sum= tensor(7.1334, device='cuda:0')
Decoder obj linear wts sum= tensor(43.1000, device='cuda:0')
Decoder obj linear bias sum= tensor(13.7138, device='cuda:0')
Epoch 56 : 0.0 % done
Current loss: 1.6664935520717077
Epoch 56 : 10.0 % done
Current loss: 9596.321313872551
Epoch 56 : 20.0 % done
C

In [27]:
torch.save(encoder.state_dict(), 'q3_encoder_weights.pt')
torch.save(decoder.state_dict(), 'q3_decoder_weights.pt')
# Add these two lines after declaring the encoder and decoder objects and before train_model()
# encoder.load_state_dict(torch.load('q3_encoder_weights.pt'))
# decoder.load_state_dict(torch.load('q3_decoder_weights.pt'))


In [28]:
def test_loss(sources, targets, enc_obj, dec_obj):
    
    loss_fn = nn.NLLLoss()
    num_sentences = len(sources)
    total_loss = 0
        
    for i in range(num_sentences):

        with torch.no_grad():

            source_sentence = torch.cuda.LongTensor(sources[i])
            target_sentence = torch.cuda.LongTensor(targets[i])
            Ts = source_sentence.size(-1)
            Tt = target_sentence.size(-1)
            enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))
            
            loss_val = 0
            
            for i in range(Ts):
                enc_output, enc_hidden = enc_obj(source_sentence[i], enc_hidden)

            dec_input = torch.tensor([[target_vocab_size-1]], device=device)  #SOS token

            # first hidden state of decoder is made the final hidden state of the encoder and cell state is initialised with zeros
            dec_hidden = (enc_hidden[0], torch.zeros(1, 1, dec_obj.hidden_size, device=device))

            for i in range(60):

                dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
                _ , index = dec_output.topk(1)
                dec_input = index.squeeze().detach()
                if index.item() == 0:
                    break

                target_word = torch.cuda.LongTensor([target_sentence[i].item()])

                loss_val += loss_fn(dec_output, target_word)
            
            total_loss += loss_val.item()/Tt
    
    return total_loss

In [29]:
# this test loss is for 500 sentences
test_loss(X_test, Y_test, encoder, decoder)

262.47782836753026

In [30]:
def eval_bleu(enc_obj, dec_obj, source_sentence, target, target_vocab_dict):
    
    # function to return the BLEU score for a single sentence 
    
    with torch.no_grad():
        
        Ts = source_sentence.size(-1)
        enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))

        for i in range(Ts):
            enc_output, enc_hidden = enc_obj(source_sentence[i], enc_hidden)

        dec_input = torch.tensor([[target_vocab_size-1]], device=device)  # SOS

        dec_hidden = (enc_hidden[0], torch.zeros(1, 1, dec_obj.hidden_size, device=device))

        predicted = []

        for i in range(60):
            dec_output, dec_hidden = dec_obj(dec_input, dec_hidden)
            _ , index = dec_output.data.topk(1)
            if index.item() == 0:
                #decoded_words.append('<EOS>')
                break
            else:
                predicted.append(target_vocab_dict[index.item()])

            dec_input = index.squeeze().detach()

    print(predicted)
    print(target)
    return bleu_score([predicted], [[target]])

In [31]:
def evaluate_model(enc_obj, dec_obj, source_test, target_test, target_vocab_dict):
    
    # returns the average bleu score for the model with the given test data
    
    total_bleu = 0
    for i in range(len(source_test)):
        source_sentence = torch.cuda.LongTensor(source_test[i])
        target = [target_vocab_dict[x] for x in target_test[i][:-1]]
        bleu = eval_bleu(enc_obj, dec_obj, source_sentence, target, target_vocab_dict)
        total_bleu += bleu
    
    return total_bleu/len(source_test)

In [32]:
evaluate_model(encoder, decoder, X_test, Y_test, TAM.index_word)

['நான்']
['நீங்கள்', 'எங்களுக்கு', 'கோப்ரா', 'என்ற', 'சொல்கிறேன்', 'அல்லது', 'die', 'நீ', 'நம்', 'மாஸ்டர்', 'கொல்ல', 'அதே', 'வாள்']
['நான்']
['அவர்கள்', 'அதே', 'படுக்கை', 'அதே', 'எழுந்து', 'அதே', 'மன', 'ஒரு', 'எரிவாயு', 'நிலையம்']
['நான்']
['என்ன', 'அன்று', 'இரவு', 'என்ன', 'நடந்தது', 'மறக்க', 'ஒரே', 'ஒரு', 'கணம்', 'இருந்தது']
['நான்']
['நீல்ஸ்', 'வேண்டும்', 'அடுத்த', 'எப்படியோ', 'நான்', 'பழைய', 'கொட்டகையின்', 'வேண்டும்', 'எல்', 'செய்']
['நான்']
['செம்மை', 'எந்த', 'அதிர்ஷ்டம்', 'இருந்தது', 'அதனால்', 'அவர்கள்', 'நமக்கு', 'கேட்டார்', 'இது', 'ஒரு', 'கிராக்', 'எடுத்து']
['நான்']
['நீங்கள்', 'அவரை', 'ஒரு', 'பெயர்', 'கொடுத்து', 'இருந்தால்', 'அவர்', 'ஒருவேளை', 'நல்ல', 'என்றார்', 'நீங்கள்', 'செல்லலாம்']
['நான்']
['போன்ற', 'இல்லை', 'இந்த', 'நாட்கள்', 'ஒன்று', 'நீங்கள்', 'போகிறேன்', 'கற்று', 'கொள்ள', 'வேண்டும்']
['நான்']
['நான்', 'தான்', 'பெற', 'முடியும்', 'அந்த', 'விமானம்', '8', '00', 'வரை', 'விட்டு', 'இல்லை']
['நான்']
['நான்', 'அது', 'என்று', 'எனக்கு', 'தெரியாது', 'நான்', 'ஆனால்', 'இந்

['நான்']
['நீ', 'சோகமாக', 'இருக்கும்', 'போது', 'நீங்கள்', 'உங்கள்', 'இதயத்தில்', 'மேல்', 'நடத்த', 'என்றால்', 'அதை', 'விட்டு', 'உங்கள்', 'துக்கம்', 'எடுக்கும்']
['நான்']
['நீங்கள்', 'fucking', 'நரம்பு', 'நிறைய', 'இங்கே', 'இரவில்', 'வரை', 'காட்ட', 'ஏய்', 'ஒரு', 'பெரும்', 'உங்கள்', 'உள்ளாடைகளை', 'மகன்', 'இல்லை']
['நான்']
['ஏனெனில்', 'அதை', 'நீங்கள்', 'என்று', 'நீங்கள்', 'திறமையான', 'நீங்கள்', 'அதை', 'பார்க்க', 'உயிரோடு', 'இருக்க', 'மாட்டேன்']
['நான்']
['என்', 'பெற்றோர்கள்', 'மிகவும்', 'ஆனால்', 'அவர்கள்', 'நம்பிக்கை', 'விட்டு', 'கொடுக்க', 'தொடங்கியது', 'கூட']
['நான்']
['நீ', 'மட்டும்', 'ஒரு', 'மாதத்திற்கு', 'இனி', 'நான்', 'விட', 'இந்த', 'குழுவினர்', 'இருந்து', 'வந்தது', 'அல்போன்ஸ்', 'நடக்கிறது', 'என்று', 'இந்த', 'கதை', 'என்ன']
['நான்']
['அவர்', 'ஒரு', 'வழக்கறிஞர்', 'மற்றும்', 'அவர்', 'ஒரு', 'அலுவலகத்தை', 'கீழே', 'இருக்கு', 'அவர்', 'சரி', 'ஒரு', 'பெரிய', 'பழைய', 'புணர்கிறேன்', 'மாளிகையை', 'கிடைத்தது']
['நான்']
['ஒரு', 'பாதுகாப்பான', 'இடத்தில்', 'நேரம்', 'இருக்கும்', 'போது', 'அந்த', 

['நான்']
['நீங்கள்', 'அதை', 'பற்றி', 'மீண்டும்', 'அங்கு', 'அனைத்து', 'என்ன', 'நரகத்தில்', 'என்னை', 'சொல்ல', 'இருக்காங்க']
['நான்']
['நான்', 'நீங்கள்', 'என்னை', 'மன்னிக்க', 'வேண்டும்', 'என்பதால்', 'நீங்கள்', 'இன்று', 'வந்து', 'நினைப்பார்கள்', 'பயம்']
['நான்']
['நாம்', 'ஒரு', 'டாலர்கள்', 'வேண்டும்', 'காதல்', 'தேடும்', 'இந்த', 'அலுவலகங்கள்', 'இருந்து', 'வெளியே', 'நடைபயிற்சி']
['நான்']
['நீங்கள்', 'என்னை', 'அனுமதிக்க', 'மாட்டேன்', 'என்றால்', 'நான்', 'எப்படி', 'உங்களுக்கு', 'உதவ', 'முடியும்']
['நான்']
['முக்கிய', 'ஒன்றும்', 'ஆனால்', 'நீங்கள்', 'இனி', 'அதை', 'எடுப்பாய்', 'இருக்கும்', 'நீ', 'அங்கே', 'பார்த்தேன்']
['நான்']
['சரி', 'அந்த', 'விசயத்துல', 'நான்', 'நீ', 'இங்க', 'வர', 'இந்த', 'ராக்']
['நான்']
['நாம்', 'வெள்ளை', 'மாளிகை', 'எல்லையை', 'பெற்று', 'விட்டேன்', 'மற்றும்', '10', 'தொகுதி', 'அகற்றப்படும்']
['நான்']
['உரிமையாளர்', 'திருமதி', 'லில்லி', 'அவர்', 'இந்த', 'பெயர்', 'அவரது', 'பிளாட்', 'மாற்றப்பட்டது']
['நான்']
['அவர்', 'என்னை', 'அங்கு', 'ஒரு', 'புத்தகம்', 'பெற', 'ஒரு', 'சில', 'உதவிக

['நான்']
['மிகவும்', 'என்ன', 'நான்', 'அதே', 'கடையில்', 'அல்லது', 'அதே', 'பயன்படுத்தப்படும்', 'என்றும்', 'உள்ளது']
['நான்']
['நான்', 'ஒரு', 'உறவு', 'இருந்தது', 'அவர்', 'வன்முறை', 'இருந்தது', 'அதனால்', 'இப்போது', 'நான்', 'இல்லை']
['நான்']
['உண்மையில்', 'நான்', 'நீங்கள்', 'சொல்ல', 'நான்', 'இங்கே', 'இருக்கிறேன்', 'சாம்', 'இந்த', 'மாலை', 'பார்க்கும்', 'போது']
['நான்']
['அடிக்கடி', 'கர்மா', 'எழுந்து', 'வந்து', 'உன்னை', 'பின்னால்', 'ஆனால்', 'நான்', 'அதை', 'பார்ப்பது', 'சில', 'முட்டைகளை', 'உடைக்க', 'வேண்டும்']
['நான்']
['நான்', 'காதல்', 'மற்றும்', 'திருமணத்தில்', 'நம்பிக்கை', 'மற்றும்', 'நான்', 'உண்மையில்', 'நீங்கள்', 'ஒரு', 'நல்ல', 'பையனை', 'நம்புகிறேன்']
['நான்']
['நான்', 'இன்னும்', 'நீங்கள்', 'இங்கு', 'எந்த', 'சட்ட', 'அதிகாரம்', 'கிடைத்தது', 'அதாவது', 'எந்த', 'தாள்', 'பார்க்க', 'வேண்டாம்']
['நான்']
['நாங்கள்', 'இன்னும்', 'மிக', 'இருக்கும்', 'பங்குகள்', 'சேமிக்கலாம்', 'பிறகும்', 'நாம்', 'இங்கு', 'இல்லை', 'கிடைத்துவிட்டது']
['நான்']
['நாம்', 'சொல்ல', 'பழைய', 'மனிதன்', 'முறைதவறி', 'பிறந்த'

['நான்']
['நீ', 'என்னை', 'ஓவியம்', 'திரும்ப', 'கொண்டு', 'வரவேண்டும்', 'நீ', 'என்', 'புணர்கிறேன்', 'நேரத்தை', 'என்று']
['நான்']
['இது', 'விசித்திரமாக', 'இல்லையா', 'போது', 'நீங்கள்', 'முடியாது', 'உங்கள்', 'சொந்த', 'விரல்கள்', 'நம்புகிறீர்களா', 'இல்லையா']
['நான்']
['கீப்ஸ்', 'நல்ல', 'இருந்த', 'போது', 'நான்', 'பெற', 'வேண்டும்', 'வேறு', 'யாரையாவது', 'உங்கள்', 'கழுதை', 'பிணை', 'எடுக்க', 'கண்டுபிடிக்க']
['நான்']
['என்', 'மீட்பவர்', 'தான்', 'எழுப்பிய', 'கேள்விகள்', 'இருப்புஅச்சுறுத்தலானஇருக்கிறது', 'சிந்திக்க', 'வேண்டும்', 'ஆனால்நான்நான்கண்டேன்']
['நான்']
['நீ', 'கொடூரமான', 'விஷயங்களை', 'பார்த்து', 'வருகின்றனர்', 'ஆனால்', 'ஒவ்வொரு', 'கொண்டு', 'நீங்கள்', 'மீது', 'மழை', 'பார்க்க', 'வேண்டும்', 'என்று']
['நான்']
['ஓஹியோ', 'வேடிக்கையாக', 'மற்றும்', 'நீங்கள்', 'எப்போதும்', 'வழியாக', 'திரும்பி', 'வந்தால்', 'ஒருவேளை', 'நீங்கள்', 'எரிவாயு', 'ஐந்து', 'நிறுத்திவிடும்']
['நான்']
['நீ', 'எனக்கு', 'கீழ்', 'என்', 'செய்து', 'என்னை', 'அந்த', 'நான்', 'அழித்து', 'நீங்கள்', 'என்', 'எதிரிகள்', 'என்னை', 'த

['நான்']
['நாங்கள்', 'ஒன்றும்', 'செய்ய', 'வேண்டும்', 'ஆனால்', 'அவர்கள்', 'ஆஷெர்', 'கொல்ல', 'துணை', 'தலைவர்', 'மற்றும்', 'அவரது', 'முழு', 'தேசிய', 'பாதுகாப்பு', 'குழு']
['நான்']
['சரி', 'எனக்கு', 'இங்கு', 'ஒரு', 'காரை', 'தேடும்', 'பழைய', 'எல்', 'செய்']
['நான்']
['ஆனால்', 'எனக்கு', 'உன்னை', 'போல்', 'fellas', 'ஐந்து', 'காதல்', 'என்றால்', 'நீங்கள்', 'முடிந்தவரை', 'தொலைவில்', 'அவர்களிடம்', 'இருந்து', 'கிடைக்கும்']
['நான்']
['இது', 'பில்லி', 'சாத்தியம்', 'என்று', 'இம்', 'நீங்கள்', 'என்னை', 'தவறாக', 'டிவிடி', 'கொடுத்தது']
['நான்']
['நாங்கள்', 'மிகவும்', 'மதிப்புமிக்க', 'அல்லது', 'பொருட்களை', 'எடுக்க', 'வேண்டும்', 'பாதுகாப்பான', 'ஒரு', 'இடத்திற்கு']
['நான்']
['ஏன்', 'என்று', 'உங்களுக்கு', 'தெரியும்', 'சில', 'வளர்ச்சி', 'ஒரு', 'முயற்சி', 'முடியாது']
['நான்']
['அவள்', 'ஒரு', 'ஜோடி', 'ஆண்டுகளுக்கு', 'முன்பு', 'அவரது', 'பெற்றோர்கள்', 'இழந்தது', 'என்று', 'சில', 'விபத்தில்', 'மருந்துகள்', 'விழுந்து']
['நான்']
['நாம்', 'அதை', 'வேறு', 'யாராவது', 'கொண்டு', 'முன்பு', 'நான்', 'ஒரு', 'நல்ல', 'இடத்

['நான்']
['நாங்கள்', 'அவள்', 'இன்னும்', 'மூச்சு', 'அவரது', 'அடக்கம்', 'சென்ற', 'போது', 'மட்டும்', 'தான்', 'இருந்தது']
['நான்']
['உங்களுக்கு', 'தெரியும்', 'நான்', 'உங்கள்', 'உயிரை', 'காப்பாற்ற', 'நான்', 'உங்கள்', 'சரி', 'கிடைக்கும்']
['நான்']
['நான்', 'உங்கள்', 'எங்கள்', 'இறுதி', 'திட்டம்', 'மூலம்', 'மகிழ்ச்சி', 'இருக்கும்', 'என்று', 'நான்', 'நம்புகிறேன்']
['நான்']
['மற்றும்', 'உங்கள்', 'குடும்பம்', 'ஏற்ப', 'நீங்கள்', 'மத', 'மற்றும்', 'கடவுள்']
['நான்']
['நாம்', 'மட்டும்', '100', 'டைனமைட்', 'என்ற', 'குச்சிகள்', 'மற்றும்', 'ஒரு', 'சில', 'ஆயுதங்கள்', 'கிடைத்தது']
['நான்']
['okay', 'ஆனால்', 'அதை', 'முப்பரிமாண', 'இரு', 'wormhole', 'ஒரு', 'வட்டம்']
['நான்']
['நான்', 'இப்போது', 'அழைப்பு', 'எடுக்க', 'கிடைக்க', 'வில்லை', 'நீங்கள்', 'ஒரு', 'செய்தியை', 'விட்டு', 'ஆனால்']
['நான்']
['எல்லன்', 'முன்பு', 'வாரங்கள்', 'மீது', 'இந்த', 'பிராடி', 'பையன்', 'அவர்', 'மோசடி', 'சிறையில்', 'சட்டவிரோத', 'சூதாட்ட', 'நேரம்', 'செய்தார்']
['நான்']
['இதை', 'சொன்னது', 'யார்', 'அரபு', 'மனிதன்', 'அவன்', 'உன்', 'மீத

['நான்']
['நீங்கள்', 'அவரை', 'உங்கள்', 'முழு', 'உடைந்த', 'சரங்களை', 'கட்டி', 'களிமண்', 'சீர்', 'செய்யப்படும்']
['நான்']
['நாள்', 'மூன்று', 'மூலம்', 'அவர்', 'ஏற்கனவே', 'எப்படியாவது', 'வரை', 'உட்கார்ந்து', 'மற்றும்', 'சுவரில்']
['நான்']
['எப்படி', 'நீங்கள்', 'எப்படி', 'பாலியல்', 'இணக்க', 'தெரியும்', 'நீங்கள்', 'அதை', 'ஒப்பிட்டு', 'இல்லை', 'போது']
['நான்']
['அதை', 'நீங்கள்', 'தெளிவான', 'குழந்தைகள்', 'முக்கிய', 'இருந்தன', 'நீங்கள்', 'நோய்', 'எதிர்ப்பு', 'என்பதால்']
['நான்']
['நீங்கள்', 'எனக்கு', 'நீங்கள்', 'உங்கள்', 'சொந்த', 'கொண்டு', 'வர', 'முடியாது', 'ஒரு', 'பொருட்டல்ல', 'உள்ளது']
['நான்']
['இரண்டு', 'வகையான', 'உள்ளன', 'இந்த', 'உலகத்தில்', 'உள்ள', 'மக்கள்', 'நடவடிக்கை', 'மற்றும்', 'அனைவரும்', 'ஆண்கள்']
['நான்']
['நான்', 'அவருடைய', 'வேகமாக', 'குணமடைய', 'தெரியும்', 'ஆனால்', 'மருத்துவமனைக்கு', 'அழைத்து', 'அவருடைய', 'உடுத்தி']
['நான்']
['என்', 'தந்தை', 'நம்புவதாக', 'உலக', 'என்றால்', 'நான்', 'உண்மையில்', 'யார்', 'கண்டு', 'அவர்கள்', 'என்னை', 'நிராகரிக்க', 'நான்', 'பயத்தில்']
['நான்']
['பா

0.0

In [78]:
tamil_train[0]

'நாம் எங்கே போகிறோம் என்று.'

In [79]:
eng_train[0]

"That's where we're going."

In [80]:
Y_train[0]

[11, 63, 359, 7, 0]

In [82]:
processed_eng_train[0]

['that', "'s", 'where', 'we', "'re", 'going', '.']

In [13]:
TAM.index_word[18669]

KeyError: 18669